# OpenAI Parameters

# Overview
When making requests to OpenAI models, several parameters can be used to control the behavior and output of the model. \
Understanding these parameters helps in fine-tuning the responses to meet specific requirements, whether for generating text, answering questions, or any other use case.

For more detailed examples, refer to the official documentation [Azure OpenAI Service](https://learn.microsoft.com/en-us/azure/ai-services/openai/reference)


In [3]:
import re
import requests
import sys
import os
from openai import AzureOpenAI
import tiktoken
from dotenv import load_dotenv
load_dotenv()


client = AzureOpenAI (
    azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT"),
    api_key = os.getenv("AZURE_OPENAI_API_KEY"), 
    api_version = os.getenv("AZURE_OPENAI_API_VERSION")
) #FIXME

CHAT_COMPLETIONS_MODEL = "gpt-4o-mini"
SEED = 123

# Parameter: max_tokens
**Description**: The maximum number of tokens to generate in the completion. \
**Default**: 16 \
**Example**: max_tokens=50

The token count of your prompt plus max_tokens can't exceed the model's context length. \
Most models have a context length of 2048 tokens (except for the newest models, which support 4096). Please refer to documentation.

In [4]:
def call_openai_with_max_tokens(max_tokens):
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages = [{"role":"system", "content":"You are a helpful assistant."},
                    {"role":"user","content": "The best pet is a"}],
        max_tokens=max_tokens
    )
    return response.choices[0].message.content #FIXME

# Generate with different presence_penalty values
penalties = [16, 32, 60, 100]
for penalty in penalties:
    print(f"Max Tokens: {penalty}\n")
    print(call_openai_with_max_tokens(penalty))
    print("\n" + "-"*80 + "\n")

Max Tokens: 16

The best pet can vary depending on individual preferences, lifestyles, and living situations.

--------------------------------------------------------------------------------

Max Tokens: 32

The best pet really depends on individual preferences and lifestyles! Here are a few popular options:

1. **Dogs** - Often called "man's best friend," dogs

--------------------------------------------------------------------------------

Max Tokens: 60

The best pet can vary greatly depending on individual preferences, lifestyles, and circumstances. Some people may find a dog to be the best pet due to their loyalty and companionship, while others might prefer a cat for its independence and playful nature. Fish, birds, rabbits, guinea pigs, or even reptiles can

--------------------------------------------------------------------------------

Max Tokens: 100

The best pet for someone can vary greatly depending on individual preferences, lifestyle, and living situation. Some popula

# Parameter: temperature

**Description**: Controls the randomness of the output. Lower values make the output more deterministic, while higher values increase randomness. \
**Value Range**: 0 to 1 \
**Default Value**: 1 \
**Example**: temperature=0.7

Higher values means the model will take more risks. \
Try 0.9 for more creative applications, and 0 (argmax sampling) for ones with a well-defined answer.

---
**NOTE**: We generally recommend altering this or top_p but not both.


In [5]:
def call_openai(num_times, prompt, temperature=0.75, use_seed=False):
    for i in range(num_times):
        if use_seed:
            response = client.chat.completions.create(
                model= CHAT_COMPLETIONS_MODEL, #FIXME
                messages = [{"role":"system", "content":"You are a helpful assistant."},
                            {"role":"user","content": f"{prompt}"}], #FIXME
                max_tokens=60,
                seed=SEED,
                temperature = temperature #FIXME
            )
        else:
            response = client.chat.completions.create(
                model= CHAT_COMPLETIONS_MODEL, #FIXME
                messages = [{"role":"system", "content":"You are a helpful assistant."},
                            {"role":"user","content": f"{prompt}"}], #FIXME
                max_tokens=60,
                temperature = temperature #FIXME
            )
        print(response.choices[0].message.content)

In [6]:
# Without seed and temperature, the response is different each time
call_openai(10, 'The best pet is a ')

The best pet can vary greatly depending on individual preferences, lifestyles, and living situations. Here are a few popular options:

1. **Dogs**: Often considered loyal companions, dogs are great for active individuals or families. They require regular exercise and training.

2. **Cats**: Cats are often
The best pet can vary depending on individual preferences, lifestyles, and living situations. Here are a few popular options:

1. **Dogs**: Known for their loyalty and companionship, dogs are often called "man's best friend." They require regular exercise and social interaction.

2. **Cats**: Cats
The best pet is subjective and varies based on individual preferences, lifestyles, and living situations. Some people may prefer dogs for their loyalty and companionship, while others might choose cats for their independence and low maintenance. Small pets like hamsters or guinea pigs can be great for those with limited space, while fish
The best pet can vary greatly depending on individual 

In [7]:
# Now using a seed and 0 temperature, the response is the much more consisitent
call_openai(10, 'The best pet is a ', temperature = 0, use_seed=True)

The best pet can vary greatly depending on individual preferences, lifestyles, and living situations. Here are a few popular options:

1. **Dogs**: Known for their loyalty and companionship, dogs are great for active individuals or families. They require regular exercise and social interaction.

2. **Cats**:
The best pet can vary greatly depending on individual preferences, lifestyles, and living situations. Here are a few popular options:

1. **Dogs**: Known for their loyalty and companionship, dogs are often considered "man's best friend." They require regular exercise and social interaction.

2. **Cats**:
The best pet can vary greatly depending on individual preferences, lifestyles, and living situations. Here are a few popular options:

1. **Dogs**: Known for their loyalty and companionship, dogs are often considered "man's best friend." They require regular exercise and social interaction.

2. **Cats**:
The best pet can vary greatly depending on individual preferences, lifestyles,

# Parameter: n
**Description**: Specifies the number of completions to generate for each prompt. \
**Default Value**: 1 \
**Example**: n = 3 

---
**Note**: Because this parameter generates many completions, it can quickly consume your token quota. Use carefully and ensure that you have reasonable settings for max_tokens and stop.

In [ ]:
response = client.chat.completions.create(
    model=CHAT_COMPLETIONS_MODEL,
    messages=[{"role":"system", "content":"You are a helpful assistant."},
              {"role":"user", "content":"The best pet is a "}], 
    n=3
) #FIXME

for index, c in enumerate(response.choices):
    print(index, c.message.content)

0 The best pet is subjective and can vary depending on individual preferences, lifestyles, and needs. Some popular choices include:

- **Dogs**: Known for their loyalty and companionship, dogs are great for active individuals or families.
- **Cats**: Independent yet affectionate, cats are ideal for people who prefer low-maintenance pets.
- **Fish**: Perfect for those who appreciate beauty and tranquility without a lot of interaction.
- **Birds**: Parrots and canaries can be social and entertaining companions.
- **Rabbits**: These small mammals can be sweet and playful, making them great indoor pets.
- **Hamsters or Guinea Pigs**: Small, easy to care for, and great for kids.

Ultimately, the best pet is one that fits your lifestyle and brings you joy!
1 The best pet can vary depending on individual preferences, lifestyle, and living situation. Some popular choices include:

- **Dogs**: Known for their loyalty and companionship, dogs are great for active individuals or families who can p

# Parameter: presence_penalty
**Description**: Penalizes new tokens based on whether they appear in the text so far, encouraging the model to use new tokens. \
**Value Range**: -2.0 to 2.0 \
**Default Value**: 0 \
**Example**: presence_penalty=0.5

In [9]:
def call_openai_with_presence_penalty(presence_penalty):
    response = client.chat.completions.create(
        model=CHAT_COMPLETIONS_MODEL,
        messages = [{"role":"system", "content":"You are a helpful assistant."},
                    {"role":"user","content": "The best pet is a"}],
        max_tokens=60,
        presence_penalty=presence_penalty #FIXME
    )
    return response.choices[0].message.content #FIXME

# Generate with different presence_penalty values
penalties = [0, 0.5, 1.0, 1.5, 2.0]
for penalty in penalties:
    print(f"Presence Penalty: {penalty}\n")
    print(call_openai_with_presence_penalty(penalty))
    print("\n" + "-"*80 + "\n")

Presence Penalty: 0

The best pet depends on your lifestyle, preferences, and needs! Here are some great options:

1. **Dogs**: Known for their loyalty and companionship, dogs are great for active individuals and families. They require regular exercise and training.

2. **Cats**: Cats are generally more independent than

--------------------------------------------------------------------------------

Presence Penalty: 0.5

The best pet can vary greatly depending on individual preferences and lifestyles. For some, dogs are considered the best pets due to their loyalty, companionship, and ability to be trained. Others might prefer cats for their independence and low maintenance. Some people may find joy in more exotic pets such as birds, reptiles

--------------------------------------------------------------------------------

Presence Penalty: 1.0

The best pet really depends on individual preferences and lifestyle! Here are a few options that people often consider:

1. **Dogs**: Know

# Parameter: frequency_penalty
**Description**: Penalizes new tokens based on their existing frequency in the text so far, reducing the likelihood of repeating the same line verbatim. \
**Value Range**: -2.0 to 2.0 \
**Default Value**: 0 \
**Example**: frequency_penalty=0.5

#### Use cases to explore
1. **Compare Responses** \
Generate multiple completions to compare and choose the best response for your use case.

2. **Increase Diversity** \
Use multiple completions to get a variety of responses, which is useful in creative applications.

3. **Enhance Robustness** \
Generate multiple responses to ensure consistency and accuracy across different completions.

#### Best Practices
1. **Optimize Prompt Length** \
Keep your prompts concise but informative to ensure the model has enough context.

2. **Adjust Temperature and Top_p** \
Use these parameters to balance between deterministic and creative responses.

3. **Monitor Token Usage** \
Be mindful of the max_tokens parameter to manage costs and response length.

4. **Use Stopping Sequences** \
Define stopping sequences to control where the model should stop generating text, ensuring the output is within the desired context.

5. **Generate Multiple Completions** \
Use the n parameter to generate multiple completions and select the best one for your needs.

## Exercise 1: Exploring 'top_p' Parameter
### Description:
- The 'top_p' parameter controls nucleus sampling. Lower values make the output more focused by limiting the pool of tokens.

### Task:
- Write a function that calls the model with different 'top_p' values and observe how the output changes.


In [12]:
def call_openai_with_top_p(top_p):
    response = client.chat.completions.create(
        model= CHAT_COMPLETIONS_MODEL,  #FIXME
        messages = [{"role":"system", "content":"You are a helpful assistant."},
                    {"role":"user","content": "Describe the benefits of exercise."}],
        max_tokens=60,
        top_p=top_p #FIXME
    )
    return response.choices[0].message.content #FIXME

# Try with different values of top_p
top_p_values = [0.1, 0.3, 0.7, 1.0]
for value in top_p_values:
    print(f"Top P: {value}\n")
    print(call_openai_with_top_p(value))
    print("\n" + "-"*80 + "\n")


Top P: 0.1

Exercise offers a wide range of benefits for both physical and mental health. Here are some of the key advantages:

### Physical Health Benefits:
1. **Improved Cardiovascular Health**: Regular exercise strengthens the heart, improves circulation, and can lower blood pressure and cholesterol levels, reducing the risk of heart

--------------------------------------------------------------------------------

Top P: 0.3

Exercise offers a wide range of benefits for both physical and mental health. Here are some of the key advantages:

### Physical Health Benefits
1. **Improved Cardiovascular Health**: Regular exercise strengthens the heart, improves circulation, and helps lower blood pressure and cholesterol levels.
2. **Weight Management

--------------------------------------------------------------------------------

Top P: 0.7

Exercise offers a wide range of benefits for both physical and mental health. Here are some key advantages:

### Physical Health Benefits:
1. **Imp

## Exercise 2: Frequency Penalty Exploration
### Description:
- This exercise will explore how the 'frequency_penalty' parameter affects the model's tendency to repeat itself.

### Task:
- Generate completions using different frequency_penalty values and note the diversity in responses.



In [13]:
def call_openai_with_frequency_penalty(frequency_penalty):
    response = client.chat.completions.create(
        model=CHAT_COMPLETIONS_MODEL,
        messages = [{"role":"system", "content":"You are a helpful assistant."},
                    {"role":"user","content": "Tell me a fun fact about space."}],
        max_tokens=60,
        frequency_penalty=frequency_penalty #FIXME
    )
    return response.choices[0].message.content #FIXME 

penalties = [-1.0, 0.0, 1.0, 2.0]
for penalty in penalties:
    print(f"Frequency Penalty: {penalty}\n")
    print(call_openai_with_frequency_penalty(penalty))
    print("\n" + "-"*80 + "\n")


Frequency Penalty: -1.0

A fun fact about space is that space is completely silent! Unlike on Earth, where sound travels through air, space is a vacuum, meaning there is no air (or other medium) for sound waves to travel through. So, if you were to be in space, you'd be unable to hear sounds

--------------------------------------------------------------------------------

Frequency Penalty: 0.0

Sure! A fun fact about space is that a day on Venus is longer than a year on Venus. Venus has an extremely slow rotation on its axis, taking about 243 Earth days to complete one full turn. In contrast, it takes only about 225 Earth days to orbit the Sun. This

--------------------------------------------------------------------------------

Frequency Penalty: 1.0

One fun fact about space is that it's completely silent! Unlike on Earth, where sound travels through air, space is a vacuum with no atmosphere to carry sound waves. So, if you were floating in space and someone shouted, you wouldn’t

## Exercise 3: Multi-Completion with 'n' Parameter
### Description:
- This exercise will help students generate multiple completions for the same prompt and compare the results.

### Task:
- Generate 3 completions for the same prompt by adjusting the 'n' parameter.


In [20]:
response = client.chat.completions.create(
    model=CHAT_COMPLETIONS_MODEL, 
    messages=[{"role":"system", "content":"You are a helpful assistant."},
              {"role":"user","content": "What are 3 qualities of a great leader."}], 
    n=3, 
    max_completion_tokens=60
) #FIXME

for index, c in enumerate(response.choices):
    print(index, "---", c.message.content, "\n\n")

0 --- A great leader typically embodies several key qualities, but three fundamental ones are:

1. **Vision**: A great leader has a clear vision and the ability to articulate it in a way that inspires others. They set direction and goals, creating a roadmap for the team or organization to follow. This vision 


1 --- Great leaders often embody a variety of qualities, but three essential qualities that stand out are:

1. **Empathy**: Great leaders possess the ability to understand and connect with the feelings and perspectives of others. This quality allows them to build strong relationships, foster trust, and create a supportive environment where 


2 --- Great leaders often embody a range of qualities, but three essential ones are:

1. **Vision**: A great leader has a clear vision of where they want to go and can articulate that vision to inspire and motivate others. They understand the bigger picture and can set realistic goals that align with that vision 




## Exercise 4: Temperature vs. Deterministic Output
### Description:
- Adjust the 'temperature' parameter to explore the balance between creative and deterministic responses.

### Task:
- Generate completions with different temperature values and analyze the variance in output.


In [22]:
def call_openai_with_temperature(temperature):
    response = client.chat.completions.create(
        model=CHAT_COMPLETIONS_MODEL,
        messages = [{"role":"system", "content":"You are a helpful assistant."},
                    {"role":"user","content": "Suggest a name for a new tech startup."}],
        max_tokens=30,
        temperature=temperature #FIXME
    )
    return response.choices[0].message.content #FIXME

temperatures = [0.0, 0.3, 0.7, 1.0]
for temp in temperatures:
    print(f"Temperature: {temp}\n")
    print(call_openai_with_temperature(temp))
    print("\n" + "-"*80 + "\n")

Temperature: 0.0

Sure! Here are some suggestions for a tech startup name:

1. **TechNova**
2. **InnoSphere**
3. **ByteBridge

--------------------------------------------------------------------------------

Temperature: 0.3

Sure! Here are some suggestions for a tech startup name:

1. **TechNova**
2. **InnoSphere**
3. **ByteBridge

--------------------------------------------------------------------------------

Temperature: 0.7

Here are some suggestions for a tech startup name:

1. **InnovaTech**
2. **NexGen Solutions**
3. **TechV

--------------------------------------------------------------------------------

Temperature: 1.0

Sure! Here are some suggestions for a tech startup name:

1. NexaTech
2. InnovateHQ
3. QuantumLeap Solutions


--------------------------------------------------------------------------------

